In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

Mounted at /content/drive
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project/spectroconv
     |████████████████████████████████| 348 kB 8.1 MB/s 
     |████████████████████████████████| 132 kB 56.8 MB/s 
     |████████████████████████████████| 81 kB 12.7 MB/s 
     |████████████████████████████████| 6.2 MB 21.3 MB/s 
     |████████████████████████████████| 8.7 MB 53.2 MB/s 
     |████████████████████████████████| 79 kB 10.9 MB/s 
     |████████████████████████████████| 138 kB 74.7 MB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [4]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Sun May  1 02:27:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from models import wb_experiment

NumExpr defaulting to 2 threads.


In [11]:
n_mels = 128

config = {
  'data': {
    'train': {
        'name': f"nsynth-full-{n_mels}-train"
    },
    'val': {
        'name': f"nsynth-full-{n_mels}-val"
    }
  },
  'group': 'cnn',
  'job_type': 'sweep',
  'model': {
      'id': wb_experiment.ModelType.CNN,
      'name': 'CNN'
  },
}

# wandb's sweep config must be exact, no additional dictionary values are allowed.
wb_sweep_config = {
    'name': 'cnn-sweep',
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'method': 'random',
    'metric': {
      'name': 'f1',
      'goal': 'maximize'
    },
    'parameters': {
      'kernel_size': {
          'value': 3,
          #'values': [3, 5]
      },
      'num_channels': {
          'distribution': 'int_uniform',
          'min': 10,
          'max': 35,
      },
      'num_conv_layers': {
          #'values': [1,2]
          'value': 2
      },
      'linear_layer_size': {
          'distribution': 'int_uniform',
          'min': 80,
          'max': 150,
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 0.00005,
          'max': 0.001,
      },
      'dropout': {# probability of an element to be zeroed (0 = no dropout)
          'value': .50
          #'distribution': 'uniform',
          #'min': .40,
          #'max': .60,
      },
      'l2_reg': {
          'value': 0, # trying out no l2 regularizer
          #'distribution': 'log_uniform_values',
          #'min': 0.0000001,
          #'max': 0.01,
      },
      'optimizer': {
          'value': 'adam'
      },
      'epochs': {
          'value': 3
      },
      'batch_size': {
          'value': 100
          #'distribution': 'q_log_uniform_values',
          #'q': 4,
          #'min': 72,
          #'max': 128,
      },
      'linear_reg': {
          'value': False
      }
    }
}

config['sweep_config'] = wb_sweep_config



In [12]:
sweeper = wb_experiment.SweepExperiment(wb_config=config, login_key=True)
sweeper.run_sweep(iterations = 50, save_all = False)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: unq4werq
Sweep URL: https://wandb.ai/jakeval-colab/spectroconv-debug/sweeps/unq4werq


wandb: Agent Starting Run: 1fn8ysbo with config:
wandb: 	batch_size: 100
wandb: 	dropout: 0.5
wandb: 	epochs: 3
wandb: 	kernel_size: 3
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0009048301045551692
wandb: 	linear_layer_size: 102
wandb: 	linear_reg: False
wandb: 	num_channels: 31
wandb: 	num_conv_layers: 2
wandb: 	optimizer: adam


hub://jakeval/nsynth-full-128-train-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train-metadata
hub://jakeval/nsynth-full-128-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train
hub://jakeval/nsynth-full-128-val-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val-metadata
hub://jakeval/nsynth-full-128-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val
CNN last layer h: 32 w 17 channels: 62 = 33728


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.
